In [4]:
import csv
import yfinance as yf
import os
import gspread
from datetime import datetime, timedelta
import pandas as pd
from tqdm import tqdm
import sqlite3
import time
from collections import Counter

In [3]:
os.getcwd()

'c:\\Users\\iamch\\OneDrive\\Documents\\python\\stocks\\trading'

In [10]:
#set local path
database_path =  'c:\\Users\\iamch\\OneDrive\\Documents\\python\\stocks\\trading\\database.db'

In [3]:
# Function to create the stocks table in the SQLite database, only use for new database
def create_price_table(path, table_name):
    conn = sqlite3.connect(path)
    cursor = conn.cursor()

    cursor.execute(f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        ticker TEXT,
        date DATE,
        open REAL,
        high REAL,
        low REAL,
        close REAL,
        volume INTEGER,
        PRIMARY KEY (ticker, date)
    )
    """)

    conn.commit()
    conn.close()

In [12]:
# Function to store stock historical data in the SQLite database
def store_stock_data_in_db(ticker, database_path, table_name):
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    stock = yf.Ticker(ticker)
    hist_data = stock.history(period="max")

    for index, row in hist_data.iterrows():
        date = index.date()
        cursor.execute(f"""
        INSERT OR IGNORE INTO {table_name} (ticker, date, open, high, low, close, volume)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (ticker, date, row["Open"], row["High"], row["Low"], row["Close"], row["Volume"]))

    conn.commit()
    conn.close()

In [8]:
#create a list of all S&P500 stock tickers from wikipedia

def get_sp500_stock_list():
    table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
    return table['Symbol'].tolist()

SP500_list = get_sp500_stock_list()

for ticker in SP500_list:
    if ticker == "BRK.B":
        ticker = "BRK-B"
    if ticker == "BF.B":
        ticker = "BF.B"

In [15]:
#ONLY RUN IF FIRST TIME

#create database
create_price_table(database_path, 'price_data')

#add historical data for all S&P500
for ticker in tqdm(SP500_list, desc='Processing items', unit='item', dynamic_ncols=True):

    try:
        store_stock_data_in_db(ticker, database_path, 'price_data')
        #print(f'Successfully saved data for {ticker}')
    except:
        print(f'Failed to retrieve data for {ticker}')

Processing items:   0%|          | 0/503 [00:00<?, ?item/s]BRK.B: No timezone found, symbol may be delisted
BF.B: No price data found, symbol may be delisted (1d 1925-05-11 -> 2024-04-17)
Processing items: 100%|██████████| 503/503 [00:00<00:00, 978.53item/s]
